In [12]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from einops import rearrange
from sklearn.metrics import confusion_matrix, classification_report

# 中文显示配置（兼容英文环境）
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备：{device}")
if device.type == 'cuda':
    print(f"GPU型号：{torch.cuda.get_device_name(0)}")

# ========== 核心参数配置 ==========
PKL_3D_DIR = '/root/second/3D'  # 你的3D骨架数据路径
TARGET_FRAMES = 32  # 统一帧长
BATCH_SIZE = 8
PRETRAIN_EPOCHS = 20  # 预训练轮数
FINETUNE_EPOCHS = 30  # 微调轮数
ANOMALY_THRESHOLD = 0.8
NUM_JOINTS = 71
PRETRAIN_LR = 1e-4    # 预训练学习率
FINETUNE_LR = 1e-5    # 微调学习率（更小）

# 模型超参数
D_MODEL = 64       # 特征维度
NHEAD = 8          # 注意力头数
NUM_LAYERS = 2     # Transformer层数
DROPOUT = 0.2      # Dropout率

# 动作组定义（按类型聚类）
ACTION_GROUPS = {
    "group_swing_squat": ["arm_swing_as", "body_swing_bs", "chest_expansion_ce", "squat_sq"],
    "group_drumming_shaking": ["drumming_dr", "maracas_forward_shaking_mfs", "maracas_shaking_ms", "sing_and_clap_sac"],
    "group_pose": ["frog_pose_fg", "tree_pose_tr", "twist_pose_tw"]
}
THEME_TO_GROUP = {theme: group_name 
                 for group_name, themes in ACTION_GROUPS.items() 
                 for theme in themes}

# ========== 保存路径配置（分层目录） ==========
BASE_OUTPUT_DIR = 'st_transformer_action_ft_results'  # 基础输出目录
PRETRAIN_DIR = os.path.join(BASE_OUTPUT_DIR, 'pretrain_model')  # 预训练模型目录
FINETUNE_DIR = os.path.join(BASE_OUTPUT_DIR, 'finetune_models')  # 微调模型目录
VISUALIZATION_DIR = os.path.join(BASE_OUTPUT_DIR, 'visualizations')  # 可视化结果目录
RESULTS_DIR = os.path.join(BASE_OUTPUT_DIR, 'detection_results')  # 检测结果目录

# 创建目录（不存在则创建）
for dir_path in [BASE_OUTPUT_DIR, PRETRAIN_DIR, FINETUNE_DIR, VISUALIZATION_DIR, RESULTS_DIR]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# 预训练模型路径（指定文件名）
PRETRAIN_MODEL_PATH = os.path.join(PRETRAIN_DIR, 'drumming_shaking_pretrain_best.pth')

使用设备：cuda
GPU型号：NVIDIA L20


In [13]:
# ========== 数据集类 ==========
class SkeletonDataset(Dataset):
    """骨架数据集（支持组内归一化和帧长统一）"""
    def __init__(self, sequences, file_names, mean=None, std=None, is_train=True):
        self.sequences = sequences  # (N, T, J, C)
        self.file_names = file_names
        self.is_train = is_train
        self.mean = mean
        self.std = std
        
        # 训练集计算全局归一化统计量
        if self.is_train:
            self.mean = np.mean(self.sequences, axis=(0, 1), keepdims=True)  # (1,1,J,C)
            self.std = np.std(self.sequences, axis=(0, 1), keepdims=True) + 1e-6  # 避免除零
        
        # 数据归一化
        self.sequences_norm = (self.sequences - self.mean) / self.std

    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences_norm[idx], dtype=torch.float32)
    
    def get_original_data(self, idx):
        """获取原始数据（非归一化）"""
        return self.sequences[idx], self.file_names[idx]

# ========== 数据加载函数（单独提取第二组） ==========
def load_data_by_group(root_dir, target_frames=32):
    """加载所有组数据，并单独返回第二组（数据量大的组）"""
    group_data = {
        "group_swing_squat": {"sequences": [], "file_names": []},
        "group_drumming_shaking": {"sequences": [], "file_names": []},
        "group_pose": {"sequences": [], "file_names": []}
    }
    
    # 遍历所有PKL文件
    pkl_files = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.pkl'):
                pkl_files.append(os.path.join(root, file))
    
    print(f"找到 {len(pkl_files)} 个pkl文件，按组加载...")
    
    for pkl_path in tqdm(pkl_files, desc="加载数据"):
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        
        # 过滤3D模态
        metadata = data['metadata']
        if metadata['data_type'].split('_')[-1] != '3d':
            continue
        
        # 匹配动作组
        theme_name = metadata['theme_name']
        if theme_name in ["arm_swing_as", "body_swing_bs", "chest_expansion_ce", "squat_sq"]:
            group_name = "group_swing_squat"
        elif theme_name in ["drumming_dr", "maracas_forward_shaking_mfs", "maracas_shaking_ms", "sing_and_clap_sac"]:
            group_name = "group_drumming_shaking"  # 第二组（数据量大）
        elif theme_name in ["frog_pose_fg", "tree_pose_tr", "twist_pose_tw"]:
            group_name = "group_pose"
        else:
            continue
        
        # 处理每个样本的帧长统一
        samples = data['samples']
        for sample in samples:
            skeleton = sample['skeleton_3d']  # (T, J, C)
            T = skeleton.shape[0]
            
            # 帧长统一为target_frames
            if T > target_frames:
                indices = np.linspace(0, T-1, target_frames, dtype=int)
                skeleton_unified = skeleton[indices]
            else:
                indices = np.linspace(0, T-1, target_frames)
                skeleton_unified = np.zeros((target_frames, NUM_JOINTS, 3))
                for j in range(NUM_JOINTS):
                    for c in range(3):
                        skeleton_unified[:, j, c] = np.interp(indices, np.arange(T), skeleton[:, j, c])
            
            group_data[group_name]["sequences"].append(skeleton_unified)
            group_data[group_name]["file_names"].append(f"{os.path.basename(pkl_path)}_{theme_name}")
    
    # 为每个组划分训练/测试集（8:2）
    group_datasets = {}
    for group_name, data in group_data.items():
        sequences = np.array(data["sequences"], dtype=np.float32)
        file_names = data["file_names"]
        
        # 划分训练/测试集
        train_size = int(0.8 * len(sequences))
        test_size = len(sequences) - train_size
        train_seq, test_seq = sequences[:train_size], sequences[train_size:]
        train_names, test_names = file_names[:train_size], file_names[train_size:]
        
        # 创建Dataset
        train_dataset = SkeletonDataset(train_seq, train_names, is_train=True)
        test_dataset = SkeletonDataset(test_seq, test_names, mean=train_dataset.mean, std=train_dataset.std, is_train=False)
        
        # 创建DataLoader
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        group_datasets[group_name] = {
            "train_dataset": train_dataset,
            "test_dataset": test_dataset,
            "train_loader": train_loader,
            "test_loader": test_loader,
            "mean": train_dataset.mean,
            "std": train_dataset.std
        }
    
    # 单独提取第二组（预训练用）
    pretrain_group = "group_drumming_shaking"
    pretrain_data = {
        "train_loader": group_datasets[pretrain_group]["train_loader"],
        "test_loader": group_datasets[pretrain_group]["test_loader"],
        "mean": group_datasets[pretrain_group]["mean"],
        "std": group_datasets[pretrain_group]["std"]
    }
    
    # 待微调的组（另外两个组）
    finetune_groups = {k: v for k, v in group_datasets.items() if k != pretrain_group}
    
    print(f"\n数据加载完成：")
    print(f"- 预训练组（{pretrain_group}）：{len(group_datasets[pretrain_group]['train_dataset'])}训练样本，{len(group_datasets[pretrain_group]['test_dataset'])}测试样本")
    for group_name, data in finetune_groups.items():
        print(f"- 微调组（{group_name}）：{len(data['train_dataset'])}训练样本，{len(data['test_dataset'])}测试样本")
    
    return pretrain_data, finetune_groups

# ========== 执行数据加载 ==========
pretrain_data, finetune_groups = load_data_by_group(PKL_3D_DIR, TARGET_FRAMES)

找到 11 个pkl文件，按组加载...


加载数据: 100%|██████████| 11/11 [00:00<00:00, 172.76it/s]


数据加载完成：
- 预训练组（group_drumming_shaking）：712训练样本，179测试样本
- 微调组（group_swing_squat）：352训练样本，88测试样本
- 微调组（group_pose）：289训练样本，73测试样本


In [14]:
class SpatialAttention(nn.Module):
    """空间注意力层：捕捉单帧内关节间依赖"""
    def __init__(self, d_model, nhead=8, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        B, T, J, d = x.shape
        x_reshaped = rearrange(x, 'b t j d -> (b t) j d')  # (B*T, J, d)
        
        # 自注意力+残差连接
        attn_out, _ = self.self_attn(x_reshaped, x_reshaped, x_reshaped)
        x_reshaped = x_reshaped + self.dropout1(attn_out)
        x_reshaped = self.norm1(x_reshaped)
        
        # 前馈网络+残差连接
        ffn_out = self.ffn(x_reshaped)
        x_reshaped = x_reshaped + self.dropout2(ffn_out)
        x_reshaped = self.norm2(x_reshaped)
        
        return rearrange(x_reshaped, '(b t) j d -> b t j d', b=B, t=T)

class TemporalAttention(nn.Module):
    """时间注意力层：捕捉帧间时序依赖"""
    def __init__(self, d_model, nhead=8, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        B, T, J, d = x.shape
        x_reshaped = rearrange(x, 'b t j d -> (b j) t d')  # (B*J, T, d)
        
        # 自注意力+残差连接
        attn_out, _ = self.self_attn(x_reshaped, x_reshaped, x_reshaped)
        x_reshaped = x_reshaped + self.dropout1(attn_out)
        x_reshaped = self.norm1(x_reshaped)
        
        # 前馈网络+残差连接
        ffn_out = self.ffn(x_reshaped)
        x_reshaped = x_reshaped + self.dropout2(ffn_out)
        x_reshaped = self.norm2(x_reshaped)
        
        return rearrange(x_reshaped, '(b j) t d -> b t j d', b=B, j=J)

class STTransformerBlock(nn.Module):
    """时空Transformer块（空间+时间注意力）"""
    def __init__(self, d_model, nhead=8, dropout=0.1):
        super().__init__()
        self.spatial_attn = SpatialAttention(d_model, nhead, dropout)
        self.temporal_attn = TemporalAttention(d_model, nhead, dropout)
        self.residual_proj = nn.Linear(d_model, d_model)  # 残差投影

    def forward(self, x):
        # 空间注意力+残差
        spatial_out = self.spatial_attn(x)
        x = x + self.residual_proj(spatial_out)
        
        # 时间注意力+残差
        temporal_out = self.temporal_attn(x)
        x = x + self.residual_proj(temporal_out)
        
        return x

class STTransformerAutoencoder(nn.Module):
    """完整的ST-Transformer自编码器"""
    def __init__(self):
        super().__init__()
        # 输入投影：(B, T, J, 3) → (B, T, J, D_MODEL)
        self.input_proj = nn.Sequential(
            nn.Linear(3, D_MODEL),
            nn.LayerNorm(D_MODEL),
            nn.ReLU(),
            nn.Dropout(DROPOUT)
        )
        
        # 编码器（堆叠时空Transformer块）
        self.encoder = nn.Sequential(*[
            STTransformerBlock(D_MODEL, NHEAD, DROPOUT) for _ in range(NUM_LAYERS)
        ])
        
        # 解码器（堆叠时空Transformer块）
        self.decoder = nn.Sequential(*[
            STTransformerBlock(D_MODEL, NHEAD, DROPOUT) for _ in range(NUM_LAYERS)
        ])
        
        # 输出投影：(B, T, J, D_MODEL) → (B, T, J, 3)
        self.output_proj = nn.Sequential(
            nn.Linear(D_MODEL, D_MODEL // 2),
            nn.ReLU(),
            nn.Dropout(DROPOUT),
            nn.Linear(D_MODEL // 2, 3)
        )

    def forward(self, x):
        # 输入投影
        x = self.input_proj(x)
        # 编码
        z = self.encoder(x)
        # 解码
        x_recon = self.decoder(z)
        # 输出投影
        x_recon = self.output_proj(x_recon)
        return x_recon

    def get_reconstruction_error(self, x):
        """计算帧级/序列级异常概率（0-1）"""
        with torch.no_grad():
            x_recon = self.forward(x)
            # 帧级MSE误差：(B, T)
            frame_errors = torch.mean((x - x_recon) ** 2, dim=[2, 3])
        
        # 归一化到0-1区间
        frame_errors_np = frame_errors.cpu().numpy()
        scaler = MinMaxScaler(feature_range=(0, 1))
        frame_probs = np.zeros_like(frame_errors_np)
        
        for i in range(frame_errors_np.shape[0]):
            frame_probs[i] = scaler.fit_transform(frame_errors_np[i].reshape(-1, 1)).flatten()
        
        # 序列级异常概率（帧概率均值）
        seq_probs = np.mean(frame_probs, axis=1)
        return seq_probs, frame_probs

In [19]:
# ========== 预训练函数（用第二组数据预训练） ==========
def pretrain_with_large_group(pretrain_data):
    """用第二组（数据量大的组）预训练模型"""
    model = STTransformerAutoencoder().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=PRETRAIN_LR, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=PRETRAIN_EPOCHS)
    
    best_loss = float('inf')
    best_acc = 0.0
    print(f"\n开始用第二组（group_drumming_shaking）预训练（{PRETRAIN_EPOCHS}轮）...")
    print("="*80)
    print(f"{'Epoch':<5} {'Train Loss':<12} {'Val Loss':<12} {'Train Acc':<12} {'Val Acc':<12}")
    print("="*80)
    
    for epoch in range(PRETRAIN_EPOCHS):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch in tqdm(pretrain_data["train_loader"], desc=f"Pretrain Epoch {epoch+1}"):
            batch = batch.to(device)
            optimizer.zero_grad()
            recon_batch = model(batch)
            loss = criterion(recon_batch, batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item() * batch.size(0)
            
            # 计算准确率
            with torch.no_grad():
                _, frame_probs = model.get_reconstruction_error(batch)
                train_correct += (frame_probs < ANOMALY_THRESHOLD).sum()
                train_total += frame_probs.size
        
        avg_train_loss = train_loss / len(pretrain_data["train_loader"].dataset)
        train_acc = train_correct / train_total if train_total > 0 else 0.0
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in pretrain_data["test_loader"]:
                batch = batch.to(device)
                recon_batch = model(batch)
                val_loss += criterion(recon_batch, batch).item() * batch.size(0)
                
                _, frame_probs = model.get_reconstruction_error(batch)
                val_correct += (frame_probs < ANOMALY_THRESHOLD).sum()
                val_total += frame_probs.size
        
        avg_val_loss = val_loss / len(pretrain_data["test_loader"].dataset)
        val_acc = val_correct / val_total if val_total > 0 else 0.0
        
        scheduler.step()
        
        # 保存最优预训练模型
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            best_acc = val_acc
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
                'accuracy': best_acc,
                'mean': pretrain_data["mean"],
                'std': pretrain_data["std"],
                'hyperparameters': {
                    'd_model': D_MODEL,
                    'nhead': NHEAD,
                    'num_layers': NUM_LAYERS,
                    'dropout': DROPOUT,
                    'pretrain_lr': PRETRAIN_LR
                }
            }, PRETRAIN_MODEL_PATH)
        
        print(f"{epoch+1:<5} {avg_train_loss:<12.6f} {avg_val_loss:<12.6f} {train_acc:<12.4f} {val_acc:<12.4f}")
    
    print(f"\n预训练完成！最优验证损失：{best_loss:.6f} | 最优验证准确率：{best_acc:.4f}")
    print(f"预训练模型保存路径：{PRETRAIN_MODEL_PATH}")
    return model

# ========== 微调函数（用预训练权重微调其他组） ==========
def finetune_other_groups(finetune_groups, pretrain_model_path):
    """用第二组的预训练权重微调另外两个组"""
    finetuned_models = {}
    
    for group_name, group_data in finetune_groups.items():
        print(f"\n========== 开始微调 {group_name} ==========")
        
        # 加载预训练模型
        model = STTransformerAutoencoder().to(device)
        pretrain_ckpt = torch.load(pretrain_model_path, map_location=device)
        model.load_state_dict(pretrain_ckpt['model_state_dict'])
        
        # 可选：冻结编码器（根据需求调整）
        # for param in model.encoder.parameters():
        #     param.requires_grad = False
        
        # 定义优化器（只优化解码器或全部参数）
        criterion = nn.MSELoss()
        optimizer = optim.AdamW(model.parameters(), lr=FINETUNE_LR, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=FINETUNE_EPOCHS)
        
        best_loss = float('inf')
        best_acc = 0.0
        train_losses, val_losses = [], []
        train_accs, val_accs = [], []
        
        # 微调模型保存路径（按组命名）
        finetune_model_path = os.path.join(FINETUNE_DIR, f"{group_name}_finetuned_best.pth")
        
        print(f"\n开始微调 {group_name}（{FINETUNE_EPOCHS}轮）...")
        print("="*80)
        print(f"{'Epoch':<5} {'Train Loss':<12} {'Val Loss':<12} {'Train Acc':<12} {'Val Acc':<12}")
        print("="*80)
        
        for epoch in range(FINETUNE_EPOCHS):
            # 训练阶段
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            
            for batch in group_data["train_loader"]:
                batch = batch.to(device)
                optimizer.zero_grad()
                recon_batch = model(batch)
                loss = criterion(recon_batch, batch)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                train_loss += loss.item() * batch.size(0)
                
                # 计算准确率
                with torch.no_grad():
                    _, frame_probs = model.get_reconstruction_error(batch)
                    train_correct += (frame_probs < ANOMALY_THRESHOLD).sum()
                    train_total += frame_probs.size
            
            avg_train_loss = train_loss / len(group_data["train_dataset"])
            train_acc = train_correct / train_total if train_total > 0 else 0.0
            train_losses.append(avg_train_loss)
            train_accs.append(train_acc)
            
            # 验证阶段
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for batch in group_data["test_loader"]:
                    batch = batch.to(device)
                    recon_batch = model(batch)
                    val_loss += criterion(recon_batch, batch).item() * batch.size(0)
                    
                    _, frame_probs = model.get_reconstruction_error(batch)
                    val_correct += (frame_probs < ANOMALY_THRESHOLD).sum()
                    val_total += frame_probs.size
            
            avg_val_loss = val_loss / len(group_data["test_dataset"])
            val_acc = val_correct / val_total if val_total > 0 else 0.0
            val_losses.append(avg_val_loss)
            val_accs.append(val_acc)
            
            scheduler.step()
            
            # 保存最优微调模型
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                best_acc = val_acc
                torch.save({
                    'group_name': group_name,
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_loss': best_loss,
                    'best_accuracy': best_acc,
                    'mean': group_data["mean"],
                    'std': group_data["std"],
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'train_accs': train_accs,
                    'val_accs': val_accs,
                    'hyperparameters': {
                        'finetune_lr': FINETUNE_LR,
                        'batch_size': BATCH_SIZE,
                        'epochs': FINETUNE_EPOCHS
                    }
                }, finetune_model_path)
            
            print(f"{epoch+1:<5} {avg_train_loss:<12.6f} {avg_val_loss:<12.6f} {train_acc:<12.4f} {val_acc:<12.4f}")
        
        # 绘制训练曲线
        plot_training_curves(group_name, train_losses, val_losses, train_accs, val_accs)
        
        # 保存微调后的模型信息
        finetuned_models[group_name] = {
            "model": model,
            "model_path": finetune_model_path,
            "best_loss": best_loss,
            "best_acc": best_acc,
            "train_losses": train_losses,
            "val_losses": val_losses,
            "train_accs": train_accs,
            "val_accs": val_accs
        }
        
        print(f"\n{group_name} 微调完成！最优验证损失：{best_loss:.6f} | 最优验证准确率：{best_acc:.4f}")
        print(f"微调模型保存路径：{finetune_model_path}")
    
    return finetuned_models

# ========== 训练曲线绘制函数 ==========
def plot_training_curves(group_name, train_losses, val_losses, train_accs, val_accs):
    """绘制训练曲线（保存到可视化目录）"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 损失曲线
    ax1.plot(train_losses, label='训练损失', color='#1f77b4', linewidth=2, marker='.')
    ax1.plot(val_losses, label='验证损失', color='#ff7f0e', linewidth=2, marker='.')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('MSE损失')
    ax1.set_title(f'{group_name} 损失曲线')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 准确率曲线
    ax2.plot(train_accs, label='训练准确率', color='#2ca02c', linewidth=2, marker='.')
    ax2.plot(val_accs, label='验证准确率', color='#d62728', linewidth=2, marker='.')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('准确率')
    ax2.set_title(f'{group_name} 准确率曲线')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 保存图片
    plot_path = os.path.join(VISUALIZATION_DIR, f"{group_name}_training_curves.png")
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"训练曲线保存路径：{plot_path}")

# ========== 执行预训练+微调 ==========
# 1. 用第二组预训练
pretrain_model = pretrain_with_large_group(pretrain_data)

# 2. 微调另外两个组
finetuned_models = finetune_other_groups(finetune_groups, PRETRAIN_MODEL_PATH)


开始用第二组（group_drumming_shaking）预训练（20轮）...
Epoch Train Loss   Val Loss     Train Acc    Val Acc     


Pretrain Epoch 1:  13%|█▎        | 12/89 [00:00<00:02, 34.65it/s]

Pretrain Epoch 1: 100%|██████████| 89/89 [00:02<00:00, 34.74it/s]


1     0.340131     0.026791     0.8948       0.9153      


Pretrain Epoch 2: 100%|██████████| 89/89 [00:02<00:00, 34.83it/s]


...

训练曲线保存路径：st_transformer_action_ft_results/visualizations/group_swing_squat_training_curves.png

group_swing_squat 微调完成！最优验证损失：0.011217 | 最优验证准确率：0.8981
微调模型保存路径：st_transformer_action_ft_results/finetune_models/group_swing_squat_finetuned_best.pth

========== 开始微调 group_pose ==========

开始微调 group_pose（30轮）...
Epoch Train Loss   Val Loss     Train Acc    Val Acc     


/tmp/ipykernel_102041/1041620738.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrain_ckpt = torch.load(pretrain_model_path, map_location=device)


1     0.075439     0.015927     0.9189       0.9247      
2     0.074924     0.015189     0.9181       0.9264      
3     0.074207     0.014564     0.9154       0.9247      
4     0.074294     0.014610     0.9147       0.9272      
5     0.073734     0.014629     0.9168       0.9255      
6     0.073676     0.013941     0.9157       0.9285      
7     0.073393     0.013729     0.9177       0.9259      
8     0.073187     0.013760     0.9128       0.9289      
9     0.072932     0.013880     0.9140       0.9251      
10    0.072618     0.013458     0.9157       0.9268      
11    0.072358     0.013606     0.9159       0.9259      
12    0.072589     0.013645     0.9146       0.9229      
13    0.072352     0.013689     0.9158       0.9259      
14    0.072265     0.013371     0.9190       0.9234      
15    0.072282     0.013234     0.9165       0.9225      
16    0.072102     0.013296     0.9151       0.9221      
17    0.071995     0.013261     0.9198       0.9247      
18    0.071853

/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 25439 (\N{CJK UNIFIED IDEOGRAPH-635F}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 22833 (\N{CJK UNIFIED IDEOGRAPH-5931}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 26354 (\N{CJK UNIFIED IDEOGRAPH-66F2}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 32447 (\N{CJK UNIFIED IDEOGRAPH-7EBF}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 35757 (\N{CJK UNIFIED IDEOGRAPH-8BAD}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 32451 (\N{CJK UNIFIED IDEOGRAPH-7EC3}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_102041/1041620738.py:249: UserWarning: Glyph 39564 (\N{CJK UN

训练曲线保存路径：st_transformer_action_ft_results/visualizations/group_pose_training_curves.png

group_pose 微调完成！最优验证损失：0.012984 | 最优验证准确率：0.9212
微调模型保存路径：st_transformer_action_ft_results/finetune_models/group_pose_finetuned_best.pth


In [11]:
# ========== 异常检测函数 ==========
def detect_anomaly(group_name, model, test_dataset, test_loader):
    """对动作组进行帧级异常检测（结果保存到新路径）"""
    model.eval()
    all_seq_probs = []
    all_frame_probs = []
    all_file_names = []
    all_frame_indices = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            batch = batch.to(device)
            seq_probs, frame_probs = model.get_reconstruction_error(batch)
            
            # 获取当前batch的文件名和帧索引
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, len(test_dataset))
            batch_files = test_dataset.file_names[start_idx:end_idx]
            
            # 记录结果
            all_seq_probs.extend(seq_probs.tolist())
            all_frame_probs.extend(frame_probs.tolist())
            all_file_names.extend(batch_files)
            all_frame_indices.extend([list(range(TARGET_FRAMES)) for _ in range(len(batch_files))])
    
    # 整理结果为DataFrame
    results = []
    for i in range(len(all_file_names)):
        for frame_idx in range(TARGET_FRAMES):
            results.append({
                'file_name': all_file_names[i],
                'frame_idx': frame_idx,
                'frame_anomaly_prob': all_frame_probs[i][frame_idx],
                'sequence_anomaly_prob': all_seq_probs[i],
                'is_anomaly': 1 if all_frame_probs[i][frame_idx] >= ANOMALY_THRESHOLD else 0
            })
    
    results_df = pd.DataFrame(results)
    
    # 保存检测结果（新路径）
    results_path = os.path.join(RESULTS_DIR, f"{group_name}_anomaly_detection_results.csv")
    results_df.to_csv(results_path, index=False, encoding='utf-8')
    
    # 统计摘要
    total_frames = len(results_df)
    anomaly_frames = results_df['is_anomaly'].sum()
    anomaly_rate = (anomaly_frames / total_frames) * 100
    
    print(f"\n{group_name} 异常检测结果：")
    print(f"- 总帧数：{total_frames}")
    print(f"- 异常帧数：{anomaly_frames}（{anomaly_rate:.2f}%）")
    print(f"- 结果保存路径：{results_path}")
    
    return results_df

# ========== 详细可视化函数（更新可视化路径） ==========
def plot_training_curves(group_name, train_losses, val_losses, train_accs, val_accs):
    """绘制训练曲线（保存到可视化目录）"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 损失曲线
    ax1.plot(train_losses, label='训练损失', color='#1f77b4', linewidth=2, marker='.')
    ax1.plot(val_losses, label='验证损失', color='#ff7f0e', linewidth=2, marker='.')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('MSE损失')
    ax1.set_title(f'{group_name} 损失曲线')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 准确率曲线
    ax2.plot(train_accs, label='训练准确率', color='#2ca02c', linewidth=2, marker='.')
    ax2.plot(val_accs, label='验证准确率', color='#d62728', linewidth=2, marker='.')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('准确率')
    ax2.set_title(f'{group_name} 准确率曲线')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 保存图片（新路径）
    plot_path = os.path.join(VISUALIZATION_DIR, f"{group_name}_training_curves.png")
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"训练曲线保存路径：{plot_path}")


In [ ]:
def detect_anomaly(group_name, model, test_dataset, test_loader):
    """对动作组进行帧级异常检测"""
    model.eval()
    all_seq_probs = []
    all_frame_probs = []
    all_file_names = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(test_loader):
            batch = batch.to(device)
            seq_probs, frame_probs = model.get_reconstruction_error(batch)
            
            all_seq_probs.extend(seq_probs)
            all_frame_probs.extend(frame_probs)
            
            # 获取文件名
            start_idx = batch_idx * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, len(test_dataset))
            all_file_names.extend(test_dataset.file_names[start_idx:end_idx])
    
    # 生成检测结果DataFrame
    frame_results = []
    for seq_idx, (file_name, seq_prob, frame_probs) in enumerate(zip(all_file_names, all_seq_probs, all_frame_probs)):
        for frame_idx in range(TARGET_FRAMES):
            frame_results.append({
                'group_name': group_name,
                'file_name': file_name,
                'sequence_idx': seq_idx,
                'frame_idx': frame_idx,
                'sequence_anomaly_prob': round(seq_prob, 4),
                'frame_anomaly_prob': round(frame_probs[frame_idx], 4),
                'is_anomaly': frame_probs[frame_idx] >= ANOMALY_THRESHOLD
            })
    
    results_df = pd.DataFrame(frame_results)
    
    # 保存结果
    results_path = os.path.join(OUTPUT_DIR, f'{group_name}_anomaly_results.csv')
    results_df.to_csv(results_path, index=False, encoding='utf-8-sig')
    
    # 统计摘要
    total_frames = len(results_df)
    anomaly_frames = results_df['is_anomaly'].sum()
    anomaly_seqs = results_df.groupby('sequence_idx')['is_anomaly'].any().sum()
    
    print(f"\n{group_name} 异常检测结果：")
    print(f"总帧数：{total_frames} | 异常帧数：{anomaly_frames} ({anomaly_frames/total_frames*100:.2f}%)")
    print(f"总序列数：{len(all_seq_probs)} | 异常序列数：{anomaly_seqs} ({anomaly_seqs/len(all_seq_probs)*100:.2f}%)")
    print(f"结果保存路径：{results_path}")
    
    return results_df

# ========== 执行异常检测 ==========
all_detection_results = {}
for group_name, info in trained_models.items():
    results_df = detect_anomaly(
        group_name=group_name,
        model=info['model'],
        test_dataset=info['test_dataset'],
        test_loader=info['test_loader']
    )
    all_detection_results[group_name] = results_df

print("\n" + "="*60)
print("所有动作组训练和检测完成！")
print(f"结果保存在：{OUTPUT_DIR}")
print("="*60)

In [9]:
# %% 单元格7：详细可视化分析（多帧数+多模态对比，修复类别数不匹配问题）
def detailed_visualization_with_frames(log_data, label_names):
    """
    详细可视化分析（支持多帧数分组）：
    1. 各分组损失/准确率曲线对比
    2. 多模态+多帧数指标雷达图
    3. 混淆矩阵（按分组展示）
    4. 类别级准确率对比
    5. 分类报告热力图
    """
    # 修复字体问题：使用通用英文字体，避免中文依赖
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Helvetica', 'sans-serif']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.figsize'] = (18, 12)  # 扩大图表尺寸，适配英文标签
    plt.rcParams['font.size'] = 11

    results_2d = log_data['2d']
    results_3d = log_data['3d']
    supported_frames = log_data['params']['supported_frames']
    epochs = log_data['params']['epochs']
    epochs_range = range(1, epochs + 1)

    # 颜色方案（兼容大小写）
    colors_modality = {'2d': '#2E86AB', '3d': '#A23B72', '2D': '#2E86AB', '3D': '#A23B72'}
    colors_frame = {32: '#F18F01', 64: '#C73E1D'}

    # ========== 图1：损失曲线对比（所有分组） ==========
    plt.subplot(2, 3, 1)
    for modality, results in [('2d', results_2d), ('3d', results_3d)]:
        for frame_num, res in results.items():
            train_loss = res['train_log']['loss']
            test_loss = res['test_log']['loss']
            label = f'{modality.upper()}-{frame_num}f'
            plt.plot(epochs_range, train_loss, label=f'{label}-Train', 
                     color=colors_modality[modality], linewidth=2, 
                     linestyle='-' if frame_num == 32 else '--')
            plt.plot(epochs_range, test_loss, label=f'{label}-Test', 
                     color=colors_modality[modality], linewidth=2, 
                     linestyle=':' if frame_num == 32 else '-.')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve Comparison (All Groups)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    # ========== 图2：准确率曲线对比（所有分组） ==========
    plt.subplot(2, 3, 2)
    for modality, results in [('2d', results_2d), ('3d', results_3d)]:
        for frame_num, res in results.items():
            train_acc = res['train_log']['acc']
            test_acc = res['test_log']['acc']
            best_acc = res['best_metrics']['acc']
            best_epoch = res['best_metrics']['epoch']
            label = f'{modality.upper()}-{frame_num}f'
            # 绘制曲线
            plt.plot(epochs_range, train_acc, label=f'{label}-Train', 
                     color=colors_modality[modality], linewidth=2, 
                     linestyle='-' if frame_num == 32 else '--')
            plt.plot(epochs_range, test_acc, label=f'{label}-Test', 
                     color=colors_modality[modality], linewidth=2, 
                     linestyle=':' if frame_num == 32 else '-.')
            # 标注最佳准确率
            plt.scatter(best_epoch, best_acc, color=colors_modality[modality], s=50, zorder=5)
            plt.annotate(f'{best_acc:.3f}', 
                         xy=(best_epoch, best_acc), 
                         xytext=(best_epoch+1, best_acc-0.05),
                         color=colors_modality[modality], fontsize=8)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Curve Comparison (All Groups)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(alpha=0.3)
    plt.ylim(0, 1.05)
    plt.tight_layout()

    # ========== 图3：多指标雷达图（最佳分组对比） ==========
    plt.subplot(2, 3, 3)
    # 收集所有分组的最佳指标
    radar_data = []
    labels_radar = []
    for modality, results in [('2d', results_2d), ('3d', results_3d)]:
        for frame_num, res in results.items():
            metrics = [
                res['best_metrics']['acc'],
                res['best_metrics']['precision'],
                res['best_metrics']['recall'],
                res['best_metrics']['f1']
            ]
            radar_data.append(metrics)
            labels_radar.append(f'{modality.upper()}-{frame_num}f')
    
    # 雷达图设置（英文指标名）
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    angles = np.linspace(0, 2*np.pi, len(metrics_names), endpoint=False).tolist()
    angles += angles[:1]
    metrics_names += metrics_names[:1]

    # 绘制雷达图（修复标签格式）
    for i, (data, label) in enumerate(zip(radar_data, labels_radar)):
        data += data[:1]
        # 兼容标签格式（提取模态部分，转为小写）
        modality_part = label.split('-')[0].lower()
        color = colors_modality[modality_part]
        plt.polar(angles, data, label=label, color=color, linewidth=2, marker='o', markersize=3)
        plt.fill(angles, data, color=color, alpha=0.1)
    
    plt.xticks(angles[:-1], metrics_names[:-1], fontsize=10)
    plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ['0.2', '0.4', '0.6', '0.8', '1.0'], alpha=0.7)
    plt.ylim(0, 1.0)
    plt.title('Multi-Metric Radar Chart (All Groups)', pad=20)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=9)
    plt.tight_layout()

    # ========== 图4：混淆矩阵（最佳性能分组） ==========
    plt.subplot(2, 3, 4)
    # 找到最佳F1的分组
    best_f1 = 0.0
    best_conf = None
    best_label = ""
    best_res = None
    for modality, results in [('2d', results_2d), ('3d', results_3d)]:
        for frame_num, res in results.items():
            if res['best_metrics']['f1'] > best_f1:
                best_f1 = res['best_metrics']['f1']
                best_preds = res['best_metrics']['preds']
                best_true = res['best_metrics']['labels']
                # 获取该分组实际存在的类别ID（去重并排序）
                actual_class_ids = sorted(list(set(best_true) | set(best_preds)))
                best_conf = confusion_matrix(best_true, best_preds, labels=actual_class_ids)
                best_label = f'{modality.upper()}-{frame_num}f'
                best_res = res  # 保存最佳结果
    
    # 关键修复：按实际类别ID过滤target_names，确保一一对应
    actual_class_names = [label_names[class_id] for class_id in actual_class_ids]
    
    # 绘制最佳分组混淆矩阵
    cm_norm = best_conf.astype('float') / best_conf.sum(axis=1)[:, np.newaxis] * 100
    sns.heatmap(cm_norm, annot=best_conf, fmt='d', cmap='Blues',
                xticklabels=actual_class_names, yticklabels=actual_class_names,
                cbar_kws={'label': 'Accuracy (%)'},
                annot_kws={'fontsize': 9})
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix (Best Group: {best_label}, F1: {best_f1:.3f})')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    plt.tight_layout()

    # ========== 图5：类别级准确率对比（所有分组） ==========
    plt.subplot(2, 3, 5)
    x = np.arange(len(actual_class_names))  # 使用实际类别数
    width = 0.15  # 柱宽
    offset = -len(radar_data)*width/2  # 偏移量

    for i, (label, _) in enumerate(zip(labels_radar, radar_data)):
        # 解析分组信息（修复格式）
        modality_part = label.split('-')[0].lower()
        frame_num = int(label.split('-')[1][:-1])
        res = results_2d[frame_num] if modality_part == '2d' else results_3d[frame_num]
        group_preds = res['best_metrics']['preds']
        group_true = res['best_metrics']['labels']
        
        # 计算该分组的类别级准确率（仅包含实际存在的类别）
        class_acc = []
        for class_id in actual_class_ids:
            mask = group_true == class_id
            if mask.sum() > 0:
                acc = (group_preds[mask] == class_id).sum() / mask.sum()
            else:
                acc = 0.0
            class_acc.append(acc)
        
        # 绘制柱状图
        plt.bar(x + offset + i*width, class_acc, width, label=label, 
                color=colors_modality[modality_part], alpha=0.8)
        # 标注数值（只标注前3个分组，避免拥挤）
        if i < 3:
            for k, v in enumerate(class_acc):
                plt.text(x[k] + offset + i*width, v + 0.02, f'{v:.2f}', 
                         ha='center', va='bottom', fontsize=7)
    
    plt.xlabel('Class')
    plt.ylabel('Class Accuracy')
    plt.title('Class-Level Accuracy Comparison (All Groups)')
    plt.xticks(x, actual_class_names, rotation=45, ha='right', fontsize=9)
    plt.ylim(0, 1.1)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(alpha=0.3, axis='y')
    plt.tight_layout()

    # ========== 图6：各分组核心指标汇总 ==========
    plt.subplot(2, 3, 6)
    # 收集核心指标
    frame_nums = []
    accs_2d = []
    accs_3d = []
    for frame_num in supported_frames:
        frame_nums.append(f'{frame_num}f')  # 简化标签为"32f"/"64f"
        accs_2d.append(results_2d[frame_num]['best_metrics']['acc'] if frame_num in results_2d else 0)
        accs_3d.append(results_3d[frame_num]['best_metrics']['acc'] if frame_num in results_3d else 0)
    
    # 绘制分组对比柱状图
    x = np.arange(len(frame_nums))
    width = 0.35
    plt.bar(x - width/2, accs_2d, width, label='2D Modality', color=colors_modality['2d'], alpha=0.8)
    plt.bar(x + width/2, accs_3d, width, label='3D Modality', color=colors_modality['3d'], alpha=0.8)
    # 标注数值
    for i, v in enumerate(accs_2d):
        if v > 0:
            plt.text(i - width/2, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontsize=10)
    for i, v in enumerate(accs_3d):
        if v > 0:
            plt.text(i + width/2, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontsize=10)
    plt.xlabel('Frame Number')
    plt.ylabel('Best Accuracy')
    plt.title('Accuracy Comparison (Modality x Frames)')
    plt.xticks(x, frame_nums, fontsize=10)
    plt.ylim(0, 1.1)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3, axis='y')
    plt.tight_layout()

    # 保存综合图表
    plt.savefig('comprehensive_visualization_with_frames.png', dpi=300, bbox_inches='tight')
    plt.show()

    # ========== 额外：分类报告热力图（最佳分组） ==========
    plt.figure(figsize=(12, 6))
    # 最佳分组分类报告（使用实际类别ID和类别名）
    best_preds = best_res['best_metrics']['preds']
    best_true = best_res['best_metrics']['labels']
    
    # 生成分类报告（指定labels参数，确保与target_names匹配）
    report = classification_report(
        best_true, best_preds, 
        labels=actual_class_ids,  # 关键：指定实际存在的类别ID
        target_names=actual_class_names,  # 过滤后的类别名
        output_dict=True,
        zero_division=0  # 避免除以零警告
    )
    report_df = pd.DataFrame(report).T[:-3]  # 去除avg和total行
    report_df = report_df[['precision', 'recall', 'f1-score']]

    # 绘制热力图
    sns.heatmap(report_df, annot=True, cmap='Blues', vmin=0, vmax=1, fmt='.3f',
                annot_kws={'fontsize': 10},
                cbar_kws={'label': 'Score'})
    plt.title(f'Classification Report (Best Group: {best_label}, F1: {best_f1:.3f})', fontsize=12)
    plt.xlabel('Metrics', fontsize=11)
    plt.ylabel('Classes', fontsize=11)
    plt.xticks(rotation=0, fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    plt.savefig('best_group_classification_report.png', dpi=300, bbox_inches='tight')
    plt.show()

    # ========== 输出详细文字报告 ==========
    print("\n" + "="*80)
    print("📋 Detailed Performance Comparison Report (Modality x Frames)")
    print("="*80)
    print(f"{'Group':<12} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Sample Size':<10}")
    print("-"*80)
    for modality, results in [('2d', results_2d), ('3d', results_3d)]:
        for frame_num, res in results.items():
            group_name = f'{modality.upper()}-{frame_num}f'
            acc = res['best_metrics']['acc']
            prec = res['best_metrics']['precision']
            rec = res['best_metrics']['recall']
            f1 = res['best_metrics']['f1']
            sample_num = res['train_samples'] + res['test_samples']
            print(f"{group_name:<12} {acc:.4f}     {prec:.4f}     {rec:.4f}     {f1:.4f}     {sample_num:<10}")

    print(f"\n🏆 Best Group: {best_label}")
    print(f"  Best F1-Score: {best_f1:.4f}")
    print(f"  Best Accuracy: {best_res['best_metrics']['acc']:.4f}")
    print(f"  Actual Classes: {actual_class_names}")  # 显示实际训练的类别
    print(f"  Corresponding Model Path: {best_res['model_path']}")

    print(f"\n📁 Generated Visualization Files:")
    print(f"  1. comprehensive_visualization_with_frames.png (Comprehensive Comparison)")
    print(f"  2. best_group_classification_report.png (Best Group Classification Report)")
    print("="*80)

# 加载训练日志并执行可视化
with open('training_logs_with_frames.pkl', 'rb') as f:
    log_data = pickle.load(f)

detailed_visualization_with_frames(log_data, label_names=dataset_2d.label_names)

NameError: name 'dataset_2d' is not defined